<font size="5" color="yellow">
Forward Kinematics
</font>



<font size="2.5">
<p>The forward kinematics (FK) represents calculation of the coordinates of the tool frame (its position and orientation) given the configuration of the robot.
</p>
</font>

In [1]:
import numpy as np
import math

<table>
  <tr>
    <td>
        <img src="images/frameAssign.jpg" width = "600" height = "300"/> </td>
        <td><img src="images/DHparamsTable.jpg" width = "500" height = "300"/></td>
  </tr>
 </table>

<table>
  <tr>
    <td>
        <img src="images/dhtable.png" width = "600" height = "300"/> </td>
        <td><img src="images/dhparams-meaning.png" width = "500" height = "300"/></td>
  </tr>
 </table>

In [2]:
a2 = 1
d1, d4, d6 = 1, 1, 1
#theta = [0.1, 0.2, 0.3, 0., 0., -1.04719755]
theta = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
params = np.array([[0, a2, 0, 0, 0, 0], 
                  [np.pi/2, 0, np.pi/2, -np.pi/2, np.pi/2, 0], 
                  [d1, 0, 0, d4, 0, d6], 
                  [theta[0], theta[1], theta[2] + np.pi/2, theta[3], theta[4], theta[5]]])

In [3]:
params 

array([[ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.57079633,  0.        ,  1.57079633, -1.57079633,  1.57079633,
         0.        ],
       [ 1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         1.        ],
       [ 0.1       ,  0.2       ,  1.87079633,  0.4       ,  0.5       ,
         0.6       ]])


<table>
  <tr>
    <td>
        <img src="images/htm.png" width = "800" height = "300"/> </td>
  </tr>
 </table>

In [4]:
def homo_trans_mat(a:float, alpha:float, d:float, theta:float) -> np.ndarray:
    '''compute homogeneous transformation matrix of ith link w.r.t. (i-1)th link based on DH parameters table
    '''
    c_theta = 0 if (theta == np.pi/2 or theta == -np.pi/2) else np.cos(theta)
    c_alpha = 0 if (alpha == np.pi/2 or alpha == -np.pi/2) else np.cos(alpha)
    
    return np.array([[c_theta, -np.sin(theta)*c_alpha, np.sin(theta)*np.sin(alpha), a*c_theta], 
                     [np.sin(theta), c_theta*c_alpha, -c_theta*np.sin(alpha), a*np.sin(theta)], 
                     [0, np.sin(alpha), c_alpha, d], 
                     [0, 0, 0, 1]])

In [5]:
for i in range(6):
    print(homo_trans_mat(params[:, i][0], params[:, i][1], params[:, i][2], params[:, i][3]))

[[ 0.99500417 -0.          0.09983342  0.        ]
 [ 0.09983342  0.         -0.99500417  0.        ]
 [ 0.          1.          0.          1.        ]
 [ 0.          0.          0.          1.        ]]
[[ 0.98006658 -0.19866933  0.          0.98006658]
 [ 0.19866933  0.98006658 -0.          0.19866933]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]
[[-0.29552021 -0.          0.95533649 -0.        ]
 [ 0.95533649 -0.          0.29552021  0.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          0.          1.        ]]
[[ 0.92106099 -0.         -0.38941834  0.        ]
 [ 0.38941834  0.          0.92106099  0.        ]
 [ 0.         -1.          0.          1.        ]
 [ 0.          0.          0.          1.        ]]
[[ 0.87758256 -0.          0.47942554  0.        ]
 [ 0.47942554  0.         -0.87758256  0.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          

In [6]:
# n = degree of freedom of the robotic arm
n = 6 
T_06 = np.eye(4)
for i in range(n):
        T_06 = T_06 @ homo_trans_mat(params[:, i][0], params[:, i][1], params[:, i][2], params[:, i][3])

In [7]:
T_06

array([[-0.47878248,  0.66404257,  0.57429505,  2.42266368],
       [-0.85419181, -0.50344118, -0.13001278,  0.05544268],
       [ 0.20278976, -0.55280597,  0.80825854,  2.48635341],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [74]:
T_06[:3,3]

array([2.42266368, 0.05544268, 2.48635341])

In [8]:
R_06 = T_06[:3, :3]

In [9]:
R_06

array([[-0.47878248,  0.66404257,  0.57429505],
       [-0.85419181, -0.50344118, -0.13001278],
       [ 0.20278976, -0.55280597,  0.80825854]])

<table>
  <tr>
        <td><img src="images/ZYZ.png" width = "600" height = "300"/></td>
  </tr>
 </table>

<font size="5" color="yellow">Case 1: When $r_{33}(q) \neq \pm 1$</font>

<table>        
<tr><td><img src="images/1.png" width = "500" height = "300"/></td></tr>
<tr><td><img src="images/2.png" width = "500" height = "300"/></td></tr>
<tr><td><img src="images/3.png" width = "500" height = "300"/></td></tr>
<tr><td><img src="images/4.png" width = "500" height = "300"/></td></tr>
</table>

<font size="5" color="yellow">Case 2: When $r_{33}(q)=1$:</font>

<font size="3">$\theta(q) = 0$, but there is uncertainty in the values of $\phi(q)$ and $\psi(q)$.</font>

<font size="5" color="yellow">Case 3: When $r_{33}(q)=-1$:</font>

<font size="3">$\theta(q) = \pi$, but there is uncertainty in the values of $\phi(q)$ and $\psi(q)$.</font>

In [10]:
if (R_06[2][2] != 1 or R_06[2][2] != -1):
    sin_thetaQ = np.sqrt(1 - R_06[2][2]**2)

    theta_1 = np.arctan2(sin_thetaQ, R_06[2][2])
    phi_1 = np.arctan2(R_06[1][2], R_06[0][2])
    psi_1 = np.arctan2(R_06[2][1], -R_06[2][0])
    print(str(T_06[:3,3][0]) + "," + str(T_06[:3,3][1]) + "," + str(T_06[:3,3][2]) + "," + str(phi_1)+','+str(theta_1)+', '+str(psi_1))
    print(str(T_06[:3,3][0]) + "," + str(T_06[:3,3][1]) + "," + str(T_06[:3,3][2]) + "," + str(math.degrees(phi_1))+','+str(math.degrees(theta_1))+', '+str(math.degrees(psi_1)))

    theta_2 = np.arctan2(-sin_thetaQ, R_06[2][2])
    phi_2 = np.arctan2(R_06[1][2], R_06[0][2])
    psi_2 = np.arctan2(R_06[2][1], R_06[2][0])
    print(str(T_06[:3,3][0]) + "," + str(T_06[:3,3][1]) + "," + str(T_06[:3,3][2]) + "," + str(phi_2)+','+str(theta_2)+', '+str(psi_2))
    print(str(T_06[:3,3][0]) + "," + str(T_06[:3,3][1]) + "," + str(T_06[:3,3][2]) + "," + str(math.degrees(phi_2))+','+str(math.degrees(theta_2))+', '+str(math.degrees(psi_2)))
else:
    print('Case leads to Uncertainty! Try a different configuration.') 

2.422663680622242,0.05544267656769389,2.4863534126490863,-0.22263396888436274,0.6296077413830932, -1.9223913750523003
2.422663680622242,0.05544267656769389,2.4863534126490863,-12.755986793320877,36.07386633001546, -110.14491236284775
2.422663680622242,0.05544267656769389,2.4863534126490863,-0.22263396888436274,-0.6296077413830932, -1.219201278537493
2.422663680622242,0.05544267656769389,2.4863534126490863,-12.755986793320877,-36.07386633001546, -69.85508763715227


In [ ]:
def testhm(q, d, a, alpha):
    c_q = 0 if (q == np.pi/2 or q == -np.pi/2) else np.cos(q)
    alpha_q = 0 if (alpha == np.pi/2 or alpha == -np.pi/2) else np.cos(alpha)

    T1 = np.array([
        [c_q, -np.sin(q), 0, 0],
        [np.sin(q), c_q, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    T2 = np.block([
        [np.eye(3), np.array([[0], [0], [d]])],
        [np.zeros((1, 3)), 1]
    ])

    T3 = np.block([
        [np.eye(3), np.array([[a], [0], [0]])],
        [np.zeros((1, 3)), 1]
    ])

    T4 = np.array([
        [1, 0, 0, 0],
        [0, alpha_q, -np.sin(alpha), 0],
        [0, np.sin(alpha), alpha_q, 0],
        [0, 0, 0, 1]
    ])

    T = T1 @ T2 @ T3 @ T4

    return T   

In [ ]:
T = np.eye(4)
for i in range(6):
    T = T @ testhm(params[:, i][3], params[:, i][2], params[:, i][0], params[:, i][1])

print(T)